In [1]:
import os
import pandas as pd
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import cv2
from collections import defaultdict

!pip install -q ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ult

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
HOME = '/content/'
!mkdir {HOME}/Detection
%cd {HOME}/Detection

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="lOVTQ88BgSlHqrup7AYi")
project = rf.workspace("dissertation-bqltf").project("apple-detection-tg6vd")
dataset = project.version(1).download("yolov8")

/content/Detection
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 26.2 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Apple-Detection-1 in yolov8:: 100%|██████████| 2014/2014 [00:01<00:00, 1919.75it/s]


In [3]:
path_dataset = '/content/Detection/Apple-Detection-1'
for ix in os.listdir(path_dataset):
  if os.path.isdir(os.path.join(path_dataset,ix)):
    s=0
    for ids in os.listdir(os.path.join(path_dataset,ix,'images')):
      if os.path.isfile(os.path.join(path_dataset,ix,'images',ids)):
        s+=1
    print(ix,s)

valid 100
test 331
train 570


## Yolov11

In [4]:
!yolo val model=yolo11n.pt data='/content/Detection/Apple-Detection-1/data.yaml' split=test imgsz=640 batch=16 conf=0.25 iou=0.6

100% 5.35M/5.35M [00:00<00:00, 148MB/s]
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs
100% 755k/755k [00:00<00:00, 73.4MB/s]
val: Scanning /content/Detection/Apple-Detection-1/test/labels... 331 images, 0 backgrounds, 0 corrupt: 100% 331/331 [00:00<00:00, 931.86it/s]
val: New cache created: /content/Detection/Apple-Detection-1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 21/21 [00:22<00:00,  1.09s/it]
                   all        331      12285          0          0          0          0
Speed: 0.7ms preprocess, 47.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [ ]:
!yolo task=detect mode=train model=yolo11n.pt data='/content/Detection/Apple-Detection-1/data.yaml' epochs=50 imgsz=720 plots=True patience=50 batch=12

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/MyDrive/Traitement images/Detection/MinneApple-floor-set-10/data.yaml, epochs=50, time=None, patience=50, batch=12, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

## Inference

In [ ]:
!yolo val model='runs/detect/train2/weights/best.pt' data='/content/Detection/Apple-Detection-1/data.yaml' split=test imgsz=640 batch=16 conf=0.25 iou=0.6

/content/drive/MyDrive/Traitement images/Detection
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Scanning /content/drive/MyDrive/Traitement images/Detection/MinneApple-floor-set-10/test/labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100% 10/10 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:00<00:00,  1.67it/s]
                   all         10        781      0.848        0.6      0.747      0.369
                 apple         10        468       0.82       0.66      0.768      0.381
        apple_on_floor          9        313      0.876       0.54      0.725      0.357
Speed: 1.0ms preprocess, 37.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val6
💡 Learn more at https://docs.ultralytics.com/modes/val


## DINO

Faire l'inférence des images test avec DINO